# Using a custom CRS with SedonaDB

Motivation is from [this thread](https://news.ycombinator.com/item?id=45362206).

**Question from czbond**: Question: Does SedonaDB support custom / alternative coordinate systems?  For example, if i wanted to define a 4d region called (fish, towel, mouse, alien) and there were floats for each of fish/towel/mouse/alien?

**Answer from paleolimbot**: SedonaDB can decode PROJJSON and authority:code CRSes at the moment, although the underlying representation is just a string. In this case you might want something like CZBOND:999 or
{ "type": "EngineeringCRS", "name": "Fish, Towel, Mouse", "datum": {"name": "Wet Kitty + Mouse In Peril"}, "coordinate_system": { "subtype": "Cartesian", "axis": [ {"name": "Fish", "abbreviation": "F", "direction": "east"}, {"name": "Towel", "abbreviation": "T", "direction": "north"}, {"name": "Mouse", "abbreviation": "M", "direction": "up"}, ] } }

(Subject to the limitations of PROJJSON, such as a 4D CRS having a temporal axis and a limited set of acceptable "direction" values)

In [6]:
paleo_crs =  { 
    "type": "EngineeringCRS", 
    "name": "Fish, Towel, Mouse", 
    "datum": {"name": "Wet Kitty + Mouse In Peril"}, 
    "coordinate_system": { "subtype": "Cartesian", "axis": [ 
        {"name": "Fish", "abbreviation": "F", "direction": "east"}, 
        {"name": "Towel", "abbreviation": "T", "direction": "north"}, 
        {"name": "Mouse", "abbreviation": "M", "direction": "up"}, 
    ] }
}

In [4]:
import sedona.db
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import json

# Initialize SedonaDB context
sd = sedona.db.connect()

In [7]:
data = {
    'id': [1, 2, 3, 4, 5],
    'feature_name': [
        'Ancient Fish Fossil',
        'Towel Deposit', 
        'Mouse Bones',             
        'Mixed Layer',
        'Surface Feature'
    ],
    'description': [
        'Early Devonian',
        'Silurian Textile',
        'Pleistocene Rodent',            
        'Ordovician Complex', 
        'Recent'
    ]
}

geometries = [
    Point(100.5, 250.3, 45.2),
    Point(150.2, 300.7, 30.1),
    Point(200.8, 275.4, 60.5),
    Point(125.6, 320.9, 25.8),
    Point(175.3, 290.1, 55.0)
]

gdf = gpd.GeoDataFrame(
    data,
    geometry=geometries,
    crs=paleo_crs
)

In [8]:
df = sd.create_data_frame(gdf)

In [9]:
df.schema

SedonaSchema with 4 fields:
  id: int64<Int64>
  feature_name: utf8<Utf8>
  description: utf8<Utf8>
  geometry: geometry<Wkb({...})>

In [10]:
df.to_view("sites")

In [12]:
df.show()

┌───────┬─────────────────────┬────────────────────┬───────────────────────────┐
│   id  ┆     feature_name    ┆     description    ┆          geometry         │
│ int64 ┆         utf8        ┆        utf8        ┆          geometry         │
╞═══════╪═════════════════════╪════════════════════╪═══════════════════════════╡
│     1 ┆ Ancient Fish Fossil ┆ Early Devonian     ┆ POINT Z(100.5 250.3 45.2) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│     2 ┆ Towel Deposit       ┆ Silurian Textile   ┆ POINT Z(150.2 300.7 30.1) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│     3 ┆ Mouse Bones         ┆ Pleistocene Rodent ┆ POINT Z(200.8 275.4 60.5) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│     4 ┆ Mixed Layer         ┆ Ordovician Complex ┆ POINT Z(125.6 320.9 25.8) │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│     5 ┆ Surface Feature   

In [15]:
sd.sql("""
SELECT 
    a.feature_name AS from_site,
    b.feature_name AS to_site,
    ROUND(ST_Distance(a.geometry, b.geometry), 2) AS distance_2d,
    ROUND(ST_X(a.geometry), 1) AS from_fish,
    ROUND(ST_Y(a.geometry), 1) AS from_towel,
    ROUND(ST_X(b.geometry), 1) AS to_fish,
    ROUND(ST_Y(b.geometry), 1) AS to_towel
FROM sites a
CROSS JOIN sites b
WHERE a.id < b.id
ORDER BY distance_2d;
""").show()

┌────────────────────┬─────────────────┬─────────────┬───────────┬────────────┬─────────┬──────────┐
│      from_site     ┆     to_site     ┆ distance_2d ┆ from_fish ┆ from_towel ┆ to_fish ┆ to_towel │
│        utf8        ┆       utf8      ┆   float64   ┆  float64  ┆   float64  ┆ float64 ┆  float64 │
╞════════════════════╪═════════════════╪═════════════╪═══════════╪════════════╪═════════╪══════════╡
│ Towel Deposit      ┆ Surface Feature ┆       27.25 ┆     150.2 ┆      300.7 ┆   175.3 ┆    290.1 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ Mouse Bones        ┆ Surface Feature ┆       29.43 ┆     200.8 ┆      275.4 ┆   175.3 ┆    290.1 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ Towel Deposit      ┆ Mixed Layer     ┆       31.83 ┆     150.2 ┆      300.7 ┆   125.6 ┆    320.9 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌